## Baseline 1
- Word-frequency inspired baseline.
- Danish stopwords were removed in a small preprocessing step rather than English stopwords.
- The code was mostly inspired by 'https://www.geeksforgeeks.org/python-text-summarizer/'.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize


def freq_summ(text):
    # Tokenizing the text
    stopWords = set(stopwords.words("danish"))
    words = word_tokenize(text)

    # Creating a frequency table to keep the
    # score of each word

    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    # Creating a dictionary to keep the score
    # of each sentence
    sentences = sent_tokenize(text)
    sentenceValue = dict()

    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq

    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]

    # Average value of a sentence from the original text

    average = int(sumValues / len(sentenceValue))

    # Storing sentences into our summary.
    summary = ''
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
            summary += " " + sentence

    return(summary.replace("\n", ""))


## Baseline 2
- PageRank inspired baseline.
- Danish was set as the language to be processed, rather than English.
- The code was mostly inspired by 'Example 16' in 'https://vimsky.com/examples/detail/python-method-networkx.pagerank.html'.

In [ ]:
from itertools import combinations
from operator import itemgetter

from distance import jaccard
from networkx import Graph, pagerank

from language import LanguageProcessor


__all__ = ['summarize']


def pr_summ(text, sentence_count=5, language='danish'):
    processor = LanguageProcessor(language)

    sentence_list = processor.split_sentences(text)
    wordset_list = map(processor.extract_significant_words, sentence_list)
    stemsets = [
        {processor.stem(word) for word in wordset}
        for wordset in wordset_list
    ]

    graph = Graph()
    pairs = combinations(enumerate(stemsets), 2)
    for (index_a, stems_a), (index_b, stems_b) in pairs:
        if stems_a and stems_b:
            similarity = 1 - jaccard(stems_a, stems_b)
            if similarity > 0:
                graph.add_edge(index_a, index_b, weight=similarity)
    ranked_sentence_indexes = list(pagerank(graph).items())
    if ranked_sentence_indexes:
        sentences_by_rank = sorted(
            ranked_sentence_indexes, key=itemgetter(1), reverse=True)
        best_sentences = map(itemgetter(0), sentences_by_rank[:sentence_count])
        best_sentences_in_order = sorted(best_sentences)
    else:
        best_sentences_in_order = range(min(sentence_count, len(sentence_list)))
    
    return ' '.join(sentence_list[index] for index in best_sentences_in_order)
    

In [ ]:
import pandas as pd

sum_dir = ""

test_df = pd.read_json(sum_dir +"data/test_bdf_da1855.json").reset_index() # The test-set remains constant across all experiments

In [ ]:
data = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
 
gen2labdf = pd.DataFrame(data, columns = ['rouge-1', 'rouge-2', 'rouge-l'], index = ["r", "p", "f"])
tot_gen2labdf = pd.DataFrame(data, columns = ['rouge-1', 'rouge-2', 'rouge-l'], index = ["r", "p", "f"])

In [ ]:
# Sampling an article for summary generation

arts = pd.read_csv('articles_summaries.csv')
article = arts['article'][0]
article

In [ ]:
freq_summ(article)

In [ ]:
pr_summ(article)

In [ ]:
def test_baseline(baseline):
    data = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
 
    gen2labdf = pd.DataFrame(data, columns = ['rouge-1', 'rouge-2', 'rouge-l'], index = ["r", "p", "f"])
    tot_gen2labdf = pd.DataFrame(data, columns = ['rouge-1', 'rouge-2', 'rouge-l'], index = ["r", "p", "f"])
    recall1_list = []
    precision1_list = []
    f11_list = []
    recall2_list = []
    precision2_list = []
    f12_list = []
    recall3_list = []
    precision3_list = []
    f13_list = []
    if baseline == "freq":
        extract_summ_fun = freq_summ
    elif baseline == "pagerank":
        extract_summ_fun = pr_summ
    for i in range(len(test_df)):
        art = test_df['docs'][i]
        example_summ = test_df['sents'][i]
        summ = extract_summ_fun(art)
        if summ!='':
            genlabi = r.get_scores(summ, example_summ)
            dfgli = pd.DataFrame(genlabi[0])
            gen2labdf=dfgli
            tot_gen2labdf+=dfgli
            gen2labdf = gen2labdf
            recall1_list.append(gen2labdf.iloc[0][0])
            recall2_list.append(gen2labdf.iloc[0][1])
            recall3_list.append(gen2labdf.iloc[0][2])
            precision1_list.append(gen2labdf.iloc[1][0])
            precision2_list.append(gen2labdf.iloc[1][1])
            precision3_list.append(gen2labdf.iloc[1][2])
            f11_list.append(gen2labdf.iloc[2][0])
            f12_list.append(gen2labdf.iloc[2][1])
            f13_list.append(gen2labdf.iloc[2][2])
    return tot_gen2labdf/len(test_df)
    

In [ ]:
from rouge import Rouge
r = Rouge()

data = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
 
gen2labdf = pd.DataFrame(data, columns = ['rouge-1', 'rouge-2', 'rouge-l'], index = ["r", "p", "f"])
tot_gen2labdf = pd.DataFrame(data, columns = ['rouge-1', 'rouge-2', 'rouge-l'], index = ["r", "p", "f"])

In [ ]:
freq_rouge_cm = test_baseline("freq")
freq_rouge_cm

In [ ]:
pr_rouge_cm = test_baseline("pagerank")
pr_rouge_cm 